NOTE:- THIS CODE WAS RUNNED ON GOOGLE COLAB, THERE ARE THING WHICH WONT RUN IN SOME COMPUTER

# Import all Libraries

In [0]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms 
import torch.optim as optiom
import matplotlib.pyplot as plt
import os
import copy
import time

use_gpu = torch.cuda.is_available()
if use_gpu:
  print("Using gpu")

# Downloading the data

In [3]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
########################################################################
########################################################################
###################### SOME USEFUL FUNCTIONS ###########################
########################################################################
########################################################################
import zipfile

def removedir(path):
  """Remove all file and directory recursively"""
  if(os.path.isfile(path)):
    os.remove(path)
  elif( os.path.isdir(path) ):
    path_list = os.listdir(path)
    for path_ in path_list:
      removedir(os.path.join(path,path_) )
    os.rmdir(path)

def unzip(path, dest):
  with zipfile.ZipFile(path, 'r') as file_to_zip:
    file_to_zip.extractall(dest)


########################################################################
########################################################################
########################################################################

In [4]:
#################### Download and Unziping the Dataset ############
from torch.hub import download_url_to_file

url = "https://uce19e2f8c0909ad1d68f11ff67d.dl.dropboxusercontent.com/cd/0/get/A5AwqVXLXPvohIcREtnTIYd6SB0AHRNX0_w1m2jjuMnclIR2oJ9mbomCitSiAqrqI91zW_nlHf3XMnctHhzpTCzJvjkUhtmNGBqW_PrOABpDHg/file?_download_id=01851980180757346097510533398638316270563901102533585381019638587&_notify_domain=www.dropbox.com&dl=1"
path = "/content/drive/My Drive/AutomaticServaliance Project"
download_url_to_file(url,path)

file = "/content/drive/My Drive/AutomaticServaliance Project/tmp7qack0ts"
unzip(file,path)

# Transforming the Data

In [0]:
TRAIN = 'train'
VAL = 'val'
TEST = 'test'


data_transforms = {
    TRAIN: transforms.Compose([
                               transforms.ToTensor()
    ]),
    VAL: transforms.Compose([
                             transforms.ToTensor()
    ]),
    TEST: transforms.Compose([
                              transforms.ToTensor()
    ])
}

